In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import tensorflow as tf
import cv2
from tqdm import tqdm
import numpy as np

from architecture import shufflenet
from input_pipeline import resize_keeping_aspect_ratio, central_crop

In [ ]:
VAL_IMAGES_PATH = '/home/gpu2/hdd/dan/imagenet/val/'
DEVKIT_PATH = '/home/gpu2/hdd/dan/imagenet/ILSVRC2012_devkit_t12/'
CHECKPOINT_PATH = 'run01/model.ckpt-1661328'
USE_MOVING_AVERAGE = True
DEPTH_MULTIPLIER = '1.0'

# Get labels for validation images


In [ ]:
with open(os.path.join(DEVKIT_PATH, 'data/ILSVRC2012_validation_ground_truth.txt')) as f:
    content = f.readlines()

content = [int(s.strip()) - 1 for s in content]  
# -1 is because the network predicts labels from 0 to 999
len(content)

In [ ]:
metadata = []
for name in os.listdir(VAL_IMAGES_PATH):
    number = name.split('_')[-1].split('.')[0]
    number = int(number.lstrip())
    label = content[number - 1]
    # -1 is because the images numbered from 1 to 50000
    metadata.append((os.path.join(VAL_IMAGES_PATH, name), label))

# Load a graph

In [ ]:
tf.reset_default_graph()

raw_image = tf.placeholder(tf.uint8, [None, None, 3])

# preprocessing: resize and do central crop
MIN_DIMENSION = 256
IMAGE_SIZE = 224
image = (1.0 / 255.0) * tf.to_float(raw_image)
image = resize_keeping_aspect_ratio(image, MIN_DIMENSION)
image = central_crop(image, crop_height=IMAGE_SIZE, crop_width=IMAGE_SIZE)
image = tf.expand_dims(image, 0)

logits = shufflenet(image, is_training=False, depth_multiplier=DEPTH_MULTIPLIER)
probabilities = tf.nn.softmax(logits, axis=1)[0]

if USE_MOVING_AVERAGE:
    ema = tf.train.ExponentialMovingAverage(decay=0.995)
    variables_to_restore = ema.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)
else:
    saver = tf.train.Saver()

# Evaluate

In [ ]:
results = []
with tf.Session() as sess:
    saver.restore(sess, CHECKPOINT_PATH)
    for path, label in tqdm(metadata):
        
        image_array = cv2.imread(path)
        assert image_array.shape[2] == 3
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

        feed_dict = {raw_image: image_array}
        result = sess.run(probabilities, feed_dict)
        top_predicted_labels = np.argsort(result)[-5:]
        
        results.append((
            top_predicted_labels[-1] == label, 
            (top_predicted_labels == label).any()
        ))

In [ ]:
accuracy = sum(t[0] for t in results)/50000
top5_accuracy = sum(t[1] for t in results)/50000

print(accuracy, top5_accuracy)